In [ ]:
import pandas as pd
import re
from tqdm import tqdm
import ast

print("Загружаем данные...")
df = pd.read_csv('dataset/habr.csv', encoding='utf-8')
print("Размер исходного датасета: {len(df)}")

def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'[^а-яа-яёёa-za-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def parse_hubs(hub_str):
    if pd.isna(hub_str):
        return []
    try:
        return ast.literal_eval(hub_str)
    except:
        if isinstance(hub_str, str):
            return [hub.strip() for hub in hub_str.split(',')]
        return []

print("Очищаем тексты...")
df['hubs'] = df['hubs'].apply(parse_hubs)

text_columns = ['title', 'keywords', 'text']
for col in text_columns:
    df[f'cleaned_{col}'] = pd.Series(tqdm(
        (preprocess_text(text) for text in df[col]),
        total=len(df),
        desc=f"Обработка {col}"
    ))

df['full_text'] = (
    df['cleaned_title'] + " " +
    df['cleaned_keywords'].fillna("") + " " +
    df['cleaned_text']
)

df = df[df['full_text'].str.len() > 50]
hub_counts = df['hubs'].explode().value_counts()
valid_hubs = hub_counts[hub_counts >= 10].index
df['hubs'] = df['hubs'].apply(lambda hubs: [h for h in hubs if h in valid_hubs])
df = df[df['hubs'].map(len) >= 3]

print(f"Итоговый размер: {len(df)}")

df.to_pickle("processed_habr.pkl")
print("Данные сохранены в processed_habr.pkl")


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_score, hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import vstack
import time
import numpy as np

df = pd.read_pickle("data/processed_habr.pkl")

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['hubs'])
print(f"Хабы: {len(mlb.classes_)}, Статьи: {df.shape[0]}")

min_hub_count = 2
valid_hub_indices = y.sum(axis=0) >= min_hub_count
y_filtered = y[:, valid_hub_indices]

print(f"Исходное число хабов: {y.shape[1]}")
print(f"После фильтрации: {y_filtered.shape[1]} хабов")

mlb_filtered = MultiLabelBinarizer()
filtered_hubs = []
for hubs_list in df['hubs']:
    filtered_hub = [hub for hub in hubs_list if hub in mlb.classes_[valid_hub_indices]]
    filtered_hubs.append(filtered_hub)

y = mlb_filtered.fit_transform(filtered_hubs)

class_counts = y.sum(axis=0)
print(f"Минимальное количество примеров в классе: {class_counts.min()}")
print(f"Максимальное количество примеров в классе: {class_counts.max()}")

non_empty_indices = y.sum(axis=1) > 0
df = df[non_empty_indices]
y = y[non_empty_indices]

print(f"Статей после удаления пустых: {len(df)}")

print("Векторизация...")
start_time = time.time()

russian_stopwords = [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
    'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да'
]

vectorizer = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 1),
    min_df=1,
    max_df=0.95,
    stop_words=russian_stopwords,
    sublinear_tf=True,
    lowercase=True,
    smooth_idf=True
)

texts = df['full_text'].tolist()
X = vectorizer.fit_transform(texts)  

print(f"\nВекторизация завершена за {time.time() - start_time:.2f} сек")
print(f"Размерность: {X.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42  
)

print(f"Обучающая выборка: {X_train.shape[0]}")
print(f"Тестовая выборка: {X_test.shape[0]}")

print("Обучение модели...")
model = MultiOutputClassifier(
    LogisticRegression(
        max_iter=1000, 
        class_weight='balanced', 
        C=0.5, 
        solver='liblinear', 
        random_state=42
    ),
    n_jobs=-1
)
model.fit(X_train, y_train)

# --- ОЦЕНКА ---
y_pred = model.predict(X_test)
jaccard = jaccard_score(y_test, y_pred, average='samples')
hamming = hamming_loss(y_test, y_pred)

print(f"\n✅ Результаты:")
print(f"Jaccard Score: {jaccard:.4f}")
print(f"Hamming Loss: {hamming:.4f}")

# Дополнительная информация о предсказаниях
print(f"\n📊 Детали:")
print(f"Количество классов: {y.shape[1]}")
print(f"Предсказано не-нулей: {(y_pred.sum(axis=1) > 0).sum()} / {y_pred.shape[0]}")
print(f"Среднее количество хабов на статью: {y.sum(axis=1).mean():.2f}")

Хабы: 955, Статьи: 98064
Исходное число хабов: 955
После фильтрации: 953 хабов
Минимальное количество примеров в классе: 2
Максимальное количество примеров в классе: 13906
Статей после удаления пустых: 98064
Векторизация...

Векторизация завершена за 149.50 сек
Размерность: (98064, 3000)
Обучающая выборка: 78451
Тестовая выборка: 19613
Обучение модели...

✅ Результаты:
Jaccard Score: 0.2451
Hamming Loss: 0.0114

📊 Детали:
Количество классов: 953
Предсказано не-нулей: 19613 / 19613
Среднее количество хабов на статью: 3.90


In [2]:
import pandas as pd
import yake
import re
import os
from tqdm import tqdm

print("Загружаем processed_habr.pkl...")
if not os.path.exists("data/processed_habr.pkl"):
    raise FileNotFoundError("Файл processed_habr.pkl не найден!")

df = pd.read_pickle("data/processed_habr.pkl")
print(f"Исходный размер датасета: {len(df)}")

N_SAMPLES = 20000
if len(df) > N_SAMPLES:
    df_sample = df.head(N_SAMPLES).copy()
    print(f"Обрабатываем выборку: {len(df_sample)} записей (первые {N_SAMPLES})")
else:
    df_sample = df.copy()
    print(f"Датасет меньше {N_SAMPLES}, обрабатываем все {len(df_sample)} записей")


kw_extractor = yake.KeywordExtractor(
    lan="ru",
    n=3,
    top=15,
    dedupLim=0.7,
    features=None
)

russian_stopwords = {
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
    'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'у', 'же',
    'бы', 'для', 'по', 'о', 'от', 'из', 'к', 'об', 'при', 'над', 'под'
}

def extractKeys(text):
    """Извлекает ключевые фразы с помощью YAKE."""
    if not isinstance(text, str) or len(text.strip()) < 10:
        return []
    try:
        keywords = kw_extractor.extract_keywords(text)
        result = []
        for phrase, score in keywords:
            words = phrase.lower().split()
            if any(word not in russian_stopwords for word in words):
                result.append(phrase.strip())
        return result
    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        return []

print("Извлекаем ключевые слова из cleaned_text с помощью YAKE...")
df_sample['text_main'] = pd.Series(tqdm(
    (extractKeys(text) for text in df_sample['cleaned_text']),
    total=len(df_sample),
    desc="YAKE: извлечение ключевых фраз",
    unit="текст"
))

print("Сохраняем промежуточный результат после YAKE...")
df_sample.to_pickle("data/sample_20k_yake_extracted.pkl")
print("Сохранено: data/sample_20k_yake_extracted.pkl")

print("Очищаем извлечённые ключевые фразы...")

def clean_keywords(keywords):
    if not isinstance(keywords, list):
        return []
    
    if not keywords:
        return []
    
    cleaned = []
    for phrase in keywords:
        if not isinstance(phrase, str):
            phrase = str(phrase)
        
        phrase = re.sub(r'[^а-яА-Яёёa-zA-Z0-9\s]', ' ', phrase)
        phrase = re.sub(r'\s+', ' ', phrase).strip()
        if phrase:
            cleaned.append(phrase)
    return cleaned

df_sample['text_main'] = df_sample['text_main'].apply(clean_keywords)

print("Дополнительная очистка: удаление дубликатов и нормализация...")
def final_clean(keywords):
    if not keywords:
        return []
    seen = set()
    result = []
    for k in keywords:
        if k not in seen:
            seen.add(k)
            result.append(k)
    return result

df_sample['text_main'] = df_sample['text_main'].apply(final_clean)

print("Сохраняем итоговые данные...")
df_sample.to_pickle("data/sample_20k_with_keywords.pkl")
df_sample.to_csv("data/sample_20k_keywords.csv", index=False, encoding='utf-8')


print("\nГотово!")
print(f"Обработано: {len(df_sample)} текстов")
print(f"Итоговый файл: sample_20k_with_keywords.pkl")
print(f"CSV-версия: sample_20k_keywords.csv")

print(f"\nПример первых 15 значений 'text_main':")
for i in range(15):
    if i < len(df_sample):
        print(f"{i}: {df_sample['text_main'].iloc[i]}")


Загружаем processed_habr.pkl...
Исходный размер датасета: 98064
Обрабатываем выборку: 20000 записей (первые 20000)
Извлекаем ключевые слова из cleaned_text с помощью YAKE...


YAKE: извлечение ключевых фраз: 100%|██████████| 20000/20000 [56:15<00:00,  5.92текст/s]  


Сохраняем промежуточный результат после YAKE...
Сохранено: sample_20k_yake_extracted.pkl
Очищаем извлечённые ключевые фразы...
Дополнительная очистка: удаление дубликатов и нормализация...
Сохраняем итоговые данные...

Готово!
Обработано: 20000 текстов
Итоговый файл: sample_20k_with_keywords.pkl
CSV-версия: sample_20k_keywords.csv

Пример первых 15 значений 'text_main':
0: ['компанию либо самый', 'либо самый умный', 'самый умный либо', 'умный либо самый', 'либо самый глупый', 'википедия небольшая компания', 'википедия небогатая компания', 'статей включая разделы', 'например шведская википедия', 'языков бла бла', 'исчезающе малы например', 'малы например родная', 'целью набрать статей', 'подавляющая часть статей', 'миллиардах просмотров статей']
1: ['apple newton messagepad', 'дополненной реальности google', 'продукт компании google', 'очки google glass', 'реальности google glassgoogle', 'смарт очки google', 'университета apple newton', 'glass продукт компании', 'производства dvd могла'

In [49]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_score, hamming_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.sparse import hstack  
import time

df = pd.read_pickle("data/sample_20k_with_keywords.pkl")

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['hubs'])
valid_hub_mask = y.sum(axis=0) >= 2 
y_filtered = y[:, valid_hub_mask]

filtered_hubs = [
    [hub for hub in hubs if hub in mlb.classes_[valid_hub_mask]]
    for hubs in df['hubs']
]
mlb_filtered = MultiLabelBinarizer()
y = mlb_filtered.fit_transform(filtered_hubs)

non_empty = y.sum(axis=1) > 0
df = df[non_empty].copy()
y = y[non_empty]

print(f"Статей после фильтрации: {len(df)}, хабов: {y.shape[1]}")

print("Векторизация...")
start_time = time.time()

df['text_main_str'] = df['text_main'].apply(lambda x: ' '.join(x) if x else '')
df['cleaned_keywords_str'] = df['cleaned_keywords'].fillna('').astype(str)
df['full_text_with_yake'] = (
    df['cleaned_title'].fillna('') + ' ' +
    df['cleaned_keywords_str'] + ' ' +
    df['text_main_str']
)

mask = df['full_text_with_yake'].str.len() > 50
df = df[mask]
y = y[mask.values] 

vectorizer = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1, 4),
    min_df=3,
    max_df=0.75,
    stop_words=[
        'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
        'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да'
    ],
    sublinear_tf=True,
    lowercase=True,
    smooth_idf=True,
    norm='l2'
)
X_text = vectorizer.fit_transform(df['full_text_with_yake'])

username_vectorizer = TfidfVectorizer(
    analyzer='char_wb',      
    ngram_range=(2, 4),    
    min_df=1,             
    max_features=500,     
    sublinear_tf=True,
    lowercase=True
)
X_username = username_vectorizer.fit_transform(df['username'].fillna(''))

X = hstack([X_text, X_username])  

print(f"!Векторизация за {time.time() - start_time:.2f} сек, X: {X.shape}, y: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Обучение модели...")
modelNew = MultiOutputClassifier(
    LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        C=0.5,
        solver='liblinear',
        random_state=42
    ),
    n_jobs=-1
)
modelNew.fit(X_train, y_train)

y_pred = modelNew.predict(X_test)
jaccard = jaccard_score(y_test, y_pred, average='samples')
hamming = hamming_loss(y_test, y_pred)

print(f"\n✅ Результаты:")
print(f"Jaccard Score: {jaccard:.4f}")
print(f"Hamming Loss:  {hamming:.4f}")
print(f"Классов: {y.shape[1]}, статей: {len(df)}")


Статей после фильтрации: 20000, хабов: 698
Векторизация...
!Векторизация за 9.94 сек, X: (19764, 20500), y: (19764, 698)
Обучение модели...

✅ Результаты:
Jaccard Score: 0.3788
Hamming Loss:  0.0073
Классов: 698, статей: 19764


In [1]:
import pandas as pd
import yake
import re
import os
from tqdm import tqdm

print("Загружаем processed_habr.pkl...")
if not os.path.exists("data/processed_habr.pkl"):
    raise FileNotFoundError("Файл processed_habr.pkl не найден!")

df = pd.read_pickle("data/processed_habr.pkl")
print(f"Исходный размер датасета: {len(df)}")

df_sample = df.copy()

kw_extractor = yake.KeywordExtractor(
    lan="ru",
    n=3,
    top=15,
    dedupLim=0.7,
    features=None
)

russian_stopwords = {
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
    'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'у', 'же',
    'бы', 'для', 'по', 'о', 'от', 'из', 'к', 'об', 'при', 'над', 'под'
}

def extractKeys(text):
    if not isinstance(text, str) or len(text.strip()) < 10:
        return []
    try:
        keywords = kw_extractor.extract_keywords(text)
        result = []
        for phrase, score in keywords:
            words = phrase.lower().split()
            if any(word not in russian_stopwords for word in words):
                result.append(phrase.strip())
        return result
    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        return []

print("Извлекаем ключевые слова из cleaned_text с помощью YAKE...")
df_sample['text_main'] = pd.Series(tqdm(
    (extractKeys(text) for text in df_sample['cleaned_text']),
    total=len(df_sample),
    desc="YAKE: извлечение ключевых фраз",
    unit="текст"
))

print("Сохраняем промежуточный результат после YAKE...")
df_sample.to_pickle("data_with_main_info_extract.pkl")
print("Сохранено: data_with_main_info_extract.pkl")

print("Очищаем извлечённые ключевые фразы...")

def clean_keywords(keywords):
    if not isinstance(keywords, list):
        return []
    
    if not keywords:
        return []
    
    cleaned = []
    for phrase in keywords:
        if not isinstance(phrase, str):
            phrase = str(phrase)
        
        phrase = re.sub(r'[^а-яА-Яёёa-zA-Z0-9\s]', ' ', phrase)
        phrase = re.sub(r'\s+', ' ', phrase).strip()
        if phrase:
            cleaned.append(phrase)
    return cleaned

df_sample['text_main'] = df_sample['text_main'].apply(clean_keywords)

print("Дополнительная очистка: удаление дубликатов и нормализация...")
def final_clean(keywords):
    if not keywords:
        return []
    seen = set()
    result = []
    for k in keywords:
        if k not in seen:
            seen.add(k)
            result.append(k)
    return result

df_sample['text_main'] = df_sample['text_main'].apply(final_clean)

print("Сохраняем итоговые данные...")
df_sample.to_pickle("data_with_main_info.pkl")
df_sample.to_csv("data_with_main_info.csv", index=False, encoding='utf-8')


print("\nГотово!")
print(f"Обработано: {len(df_sample)} текстов")
print(f"Итоговый файл: data_with_main_info.pkl")
print(f"CSV-версия: data_with_main_info.csv")

print(f"\nПример первых 15 значений 'text_main':")
for i in range(15):
    if i < len(df_sample):
        print(f"{i}: {df_sample['text_main'].iloc[i]}")


Загружаем processed_habr.pkl...
Исходный размер датасета: 98064
Извлекаем ключевые слова из cleaned_text с помощью YAKE...


YAKE: извлечение ключевых фраз: 100%|██████████| 98064/98064 [4:36:28<00:00,  5.91текст/s]   


Сохраняем промежуточный результат после YAKE...
Сохранено: data_with_main_info_extract.pkl
Очищаем извлечённые ключевые фразы...
Дополнительная очистка: удаление дубликатов и нормализация...
Сохраняем итоговые данные...

Готово!
Обработано: 98064 текстов
Итоговый файл: data_with_main_info.pkl
CSV-версия: data_with_main_info.csv

Пример первых 15 значений 'text_main':
0: ['компанию либо самый', 'либо самый умный', 'самый умный либо', 'умный либо самый', 'либо самый глупый', 'википедия небольшая компания', 'википедия небогатая компания', 'статей включая разделы', 'например шведская википедия', 'языков бла бла', 'исчезающе малы например', 'малы например родная', 'целью набрать статей', 'подавляющая часть статей', 'миллиардах просмотров статей']
1: ['apple newton messagepad', 'дополненной реальности google', 'продукт компании google', 'очки google glass', 'реальности google glassgoogle', 'смарт очки google', 'университета apple newton', 'glass продукт компании', 'производства dvd могла', '

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_score, hamming_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
from scipy.sparse import hstack 
import time
import numpy as np
import joblib  

df = pd.read_pickle("data/data_with_main_info.pkl")

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['hubs'])
valid_hub_mask = y.sum(axis=0) >= 2  
y_filtered = y[:, valid_hub_mask]

filtered_hubs = [
    [hub for hub in hubs if hub in mlb.classes_[valid_hub_mask]]
    for hubs in df['hubs']
]
mlb_filtered = MultiLabelBinarizer()
y = mlb_filtered.fit_transform(filtered_hubs)

non_empty = y.sum(axis=1) > 0
df = df[non_empty].copy()
y = y[non_empty]

print(f"Статей после фильтрации: {len(df)}, хабов: {y.shape[1]}")

print("Векторизация...")
start_time = time.time()

df['text_main_str'] = df['text_main'].apply(lambda x: ' '.join(x) if x else '')
df['cleaned_keywords_str'] = df['cleaned_keywords'].fillna('').astype(str)
df['full_text_with_yake'] = (
    df['cleaned_title'].fillna('') + ' ' +
    df['cleaned_keywords_str'] + ' ' +
    df['text_main_str']
)

mask = df['full_text_with_yake'].str.len() > 50
df = df[mask]
y = y[mask.values] 

df['datetime'] = pd.to_datetime(df['time'])
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

time_features = ['year', 'month', 'dayofweek', 'hour']
time_encoder = OneHotEncoder(sparse_output=True, drop='first')
X_time = time_encoder.fit_transform(df[time_features])

vectorizer = TfidfVectorizer(
    max_features=40000,           
    ngram_range=(1, 3),           
    min_df=2,                     
    max_df=0.8,                
    stop_words=[
        'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
        'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да'
    ],
    sublinear_tf=True,
    lowercase=True,
    smooth_idf=True,
    norm='l2',
    use_idf=True,
    binary=False
)
X_text = vectorizer.fit_transform(df['full_text_with_yake'])

username_vectorizer = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 4),
    min_df=1,
    max_features=1500,
    sublinear_tf=True,
    lowercase=True
)
X_username = username_vectorizer.fit_transform(df['username'].fillna(''))

X = hstack([X_text, X_username, X_time])

print(f"!Векторизация за {time.time() - start_time:.2f} сек, X: {X.shape}, y: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Обучение модели...")
modelFull = MultiOutputClassifier(
    LogisticRegression(
        max_iter=2000,
        class_weight='balanced',
        C=0.5,
        solver='liblinear',
        random_state=42,
        tol=1e-4,
        fit_intercept=True
    ),
    n_jobs=-1
)
modelFull.fit(X_train, y_train)

y_pred = modelFull.predict(X_test)
jaccard = jaccard_score(y_test, y_pred, average='samples')
hamming = hamming_loss(y_test, y_pred)

print(f"\n✅ Результаты:")
print(f"Jaccard Score: {jaccard:.4f}")
print(f"Hamming Loss:  {hamming:.4f}")
print(f"Классов: {y.shape[1]}, статей: {len(df)}")

joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(username_vectorizer, 'username_vectorizer.pkl')
joblib.dump(time_encoder, 'time_encoder.pkl')
joblib.dump(mlb_filtered, 'mlb_filtered.pkl')
joblib.dump(modelFull, 'helpers/modelFull.pkl')  


Статей после фильтрации: 98064, хабов: 953
Векторизация...
!Векторизация за 43.58 сек, X: (97731, 41549), y: (97731, 953)
Обучение модели...

✅ Результаты:
Jaccard Score: 0.3700
Hamming Loss:  0.0060
Классов: 953, статей: 97731


['modelFull.pkl']

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_score, hamming_loss, f1_score, precision_score, recall_score
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder, StandardScaler
from scipy.sparse import hstack, csr_matrix
import time
import numpy as np
import joblib
from collections import Counter

df = pd.read_pickle("data/data_with_main_info.pkl")

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['hubs'])

min_samples_per_class = max(3, int(len(df) * 0.001))  
valid_hub_mask = y.sum(axis=0) >= min_samples_per_class
y_filtered = y[:, valid_hub_mask]

filtered_hubs = [
    [hub for hub in hubs if hub in mlb.classes_[valid_hub_mask]]
    for hubs in df['hubs']
]
mlb_filtered = MultiLabelBinarizer()
y = mlb_filtered.fit_transform(filtered_hubs)

non_empty = y.sum(axis=1) > 0
df = df[non_empty].copy()
y = y[non_empty]

print(f"Статей после фильтрации: {len(df)}, хабов: {y.shape[1]}")
print(f"Распределение хабов: {dict(Counter(y.sum(axis=1)).most_common(10))}")

print("\nВекторизация...")
start_time = time.time()

df['text_main_str'] = df['text_main'].apply(lambda x: ' '.join(x) if x else '')
df['cleaned_keywords_str'] = df['cleaned_keywords'].fillna('').astype(str)

df['full_text_weighted'] = (
    df['cleaned_title'].fillna('') + ' ' +
    df['cleaned_title'].fillna('') + ' ' +
    df['cleaned_keywords_str'] + ' ' +
    df['text_main_str']
)

min_text_length = 50
mask = df['full_text_weighted'].str.len() > min_text_length
df = df[mask]
y = y[mask.values]

df['datetime'] = pd.to_datetime(df['time'])
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour
df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
df['quarter'] = df['datetime'].dt.quarter

time_features = ['year', 'month', 'dayofweek', 'hour', 'is_weekend', 'quarter']
time_encoder = OneHotEncoder(sparse_output=True, drop='first')
X_time = time_encoder.fit_transform(df[time_features])

vectorizer = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8,
    stop_words=[
        'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со',
        'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да',
        'ты', 'по', 'но', 'за', 'из', 'это', 'или', 'у', 'же', 'бы',
        'вот', 'от', 'меня', 'ему', 'нет', 'о', 'еще', 'когда',
        'даже', 'ну', 'ли', 'если', 'был', 'до', 'ни', 'быть',
        'при', 'также', 'к', 'по', 'на', 'этот', 'что', 'который'
    ],
    sublinear_tf=True,
    lowercase=True,
    smooth_idf=True,
    norm='l2',
    use_idf=True,
    binary=False,
    analyzer='word'
)

X_text = vectorizer.fit_transform(df['full_text_weighted'])

username_vectorizer = TfidfVectorizer(
    analyzer='char',
    ngram_range=(3, 4),
    min_df=2,
    max_features=1500,
    sublinear_tf=True,
    lowercase=True,
    binary=True
)
X_username = username_vectorizer.fit_transform(df['username'].fillna(''))

df['text_length'] = df['full_text_weighted'].str.len()
df['word_count'] = df['full_text_weighted'].str.split().str.len()

scaler = StandardScaler(with_mean=False)
X_numeric = csr_matrix(scaler.fit_transform(df[['text_length', 'word_count']]))

X = hstack([X_text, X_username, X_time, X_numeric])
print(f"Векторизация за {time.time() - start_time:.2f} сек, X: {X.shape}, y: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nРазмер train: {X_train.shape}, test: {X_test.shape}")

print("\nОбучение модели...")
model = MultiOutputClassifier(
    LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        C=0.8,
        solver='liblinear',
        penalty='l2',
        random_state=42,
        tol=1e-4,
        fit_intercept=True
    ),
    n_jobs=-1
)

print("КРОСС-ВАЛИДАЦИЯ (5 фолдов)")

kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_jaccard_scores = []
fold_f1_micro_scores = []
fold_f1_macro_scores = []
fold_hamming_losses = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train), 1):
    print(f"\nФолд {fold}/5:")
    
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
    
    model_fold = MultiOutputClassifier(
        LogisticRegression(
            max_iter=1000,
            class_weight='balanced',
            C=0.8,
            solver='liblinear',
            random_state=42 + fold
        ),
        n_jobs=-1
    )
    
    model_fold.fit(X_train_fold, y_train_fold)
    
    y_pred_fold = model_fold.predict(X_val_fold)
    
    jaccard_fold = jaccard_score(y_val_fold, y_pred_fold, average='samples')
    f1_micro_fold = f1_score(y_val_fold, y_pred_fold, average='micro')
    f1_macro_fold = f1_score(y_val_fold, y_pred_fold, average='macro')
    hamming_fold = hamming_loss(y_val_fold, y_pred_fold)
    
    fold_jaccard_scores.append(jaccard_fold)
    fold_f1_micro_scores.append(f1_micro_fold)
    fold_f1_macro_scores.append(f1_macro_fold)
    fold_hamming_losses.append(hamming_fold)
    
    print(f"  Jaccard: {jaccard_fold:.4f}, F1 Micro: {f1_micro_fold:.4f}, Hamming Loss: {hamming_fold:.4f}")

print("РЕЗУЛЬТАТЫ КРОСС-ВАЛИДАЦИИ")

print(f"Jaccard Score (samples):")
print(f"  Фолды: {[f'{v:.4f}' for v in fold_jaccard_scores]}")
print(f"  Среднее: {np.mean(fold_jaccard_scores):.4f} (±{np.std(fold_jaccard_scores):.4f})")

print(f"\nF1 Micro:")
print(f"  Фолды: {[f'{v:.4f}' for v in fold_f1_micro_scores]}")
print(f"  Среднее: {np.mean(fold_f1_micro_scores):.4f} (±{np.std(fold_f1_micro_scores):.4f})")

print(f"\nF1 Macro:")
print(f"  Фолды: {[f'{v:.4f}' for v in fold_f1_macro_scores]}")
print(f"  Среднее: {np.mean(fold_f1_macro_scores):.4f} (±{np.std(fold_f1_macro_scores):.4f})")

print(f"\nHamming Loss:")
print(f"  Фолды: {[f'{v:.4f}' for v in fold_hamming_losses]}")
print(f"  Среднее: {np.mean(fold_hamming_losses):.4f} (±{np.std(fold_hamming_losses):.4f})")

print("ОБУЧЕНИЕ ФИНАЛЬНОЙ МОДЕЛИ")

model.fit(X_train, y_train)

print("ОЦЕНКА НА ТЕСТОВЫХ ДАННЫХ")

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

jaccard = jaccard_score(y_test, y_pred, average='samples')
hamming = hamming_loss(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
precision_micro = precision_score(y_test, y_pred, average='micro')
recall_micro = recall_score(y_test, y_pred, average='micro')

print(f"\n✅ Результаты на тестовых данных:")
print(f"Jaccard Score (samples): {jaccard:.4f}")
print(f"Hamming Loss: {hamming:.4f}")
print(f"F1 Micro: {f1_micro:.4f}")
print(f"F1 Macro: {f1_macro:.4f}")
print(f"Precision Micro: {precision_micro:.4f}")
print(f"Recall Micro: {recall_micro:.4f}")
print(f"Классов: {y.shape[1]}, статей: {len(df)}")

print("ЭКСПЕРИМЕНТ С РАЗНЫМИ ПОРОГАМИ")

thresholds = [0.1, 0.2, 0.3, 0.4, 0.5]
for threshold in thresholds:
    y_pred_thresholded = np.array([(proba[:, 1] > threshold).astype(int) for proba in y_pred_proba]).T
    jaccard_th = jaccard_score(y_test, y_pred_thresholded, average='samples')
    print(f"Порог {threshold}: Jaccard = {jaccard_th:.4f}")

print("СОХРАНЕНИЕ МОДЕЛИ")

joblib.dump(vectorizer, 'actual_helpers/vectorizer_last.pkl')
joblib.dump(username_vectorizer, 'actual_helpers/username_vectorizer_last.pkl')
joblib.dump(time_encoder, 'actual_helpers/time_encoder_last.pkl')
joblib.dump(mlb_filtered, 'actual_helpers/mlb_filtered_last.pkl')
joblib.dump(model, 'model_last.pkl')
joblib.dump(scaler, 'actual_helpers/scaler_last.pkl')

print(f"Результаты кросс-валидации: Jaccard = {np.mean(fold_jaccard_scores):.4f} (±{np.std(fold_jaccard_scores):.4f})")
print(f"Результаты на тесте: Jaccard = {jaccard:.4f}")

Статей после фильтрации: 98061, хабов: 393
Распределение хабов: {np.int64(3): 39006, np.int64(4): 29976, np.int64(5): 23474, np.int64(2): 5465, np.int64(1): 133, np.int64(6): 7}

Векторизация...
Векторизация за 33.22 сек, X: (98035, 31555), y: (98035, 393)

Размер train: (78428, 31555), test: (19607, 31555)

Обучение модели...
КРОСС-ВАЛИДАЦИЯ (5 фолдов)

Фолд 1/5:
  Jaccard: 0.3858, F1 Micro: 0.5206, Hamming Loss: 0.0125

Фолд 2/5:
  Jaccard: 0.3843, F1 Micro: 0.5196, Hamming Loss: 0.0125

Фолд 3/5:
  Jaccard: 0.3816, F1 Micro: 0.5154, Hamming Loss: 0.0126

Фолд 4/5:
  Jaccard: 0.3841, F1 Micro: 0.5191, Hamming Loss: 0.0126

Фолд 5/5:
  Jaccard: 0.3818, F1 Micro: 0.5170, Hamming Loss: 0.0126
РЕЗУЛЬТАТЫ КРОСС-ВАЛИДАЦИИ
Jaccard Score (samples):
  Фолды: ['0.3858', '0.3843', '0.3816', '0.3841', '0.3818']
  Среднее: 0.3835 (±0.0016)

F1 Micro:
  Фолды: ['0.5206', '0.5196', '0.5154', '0.5191', '0.5170']
  Среднее: 0.5183 (±0.0019)

F1 Macro:
  Фолды: ['0.5385', '0.5386', '0.5314', '0.5382',